# ML Table Creation 1

First iteration of ML Table creation for a cleaned table ready for ML. The table will be used as reference in ML testing. The table is uploaded to the teams S3 bucket as "MLTable1.csv"

#### Method:

This is code for creating a ML which consist of satellite detections (NASA FIRMS detections) considered as instances  which were merged wwith weather and soil data from the USDA SCAN network.

The table table is then merged with WFIGS  actual records of wildifres based on initial coordinates and date the fires started. 

Certain columns were dropped for multiple resons described in the notebook.

Lastly, the nan values were dropped, giving a table ready for ML reference.

#### Additional Note:

Most FIRMS detections were not associated with a major fire. The ML activities are in attempt to train a model to predict if a FIRMS detection will lead to a fire. 

This will support the team's goal **“Given that a FIRMS (Fire Information for Resource Management System) detection is found, how might we predict if the detection will turn into a wildfire by referencing historical WFIGS (Wildland Fire
Interagency Geospatial Services) data?”.**

In [1]:
#general imports to start
#general imports
import pandas as pd
import boto3
import numpy as np
pd.set_option('display.max_columns', 200)

## Load and Merge the Data

In [2]:
#load in the csvs
#TODO For Team: enter the credentails below to run
S3_Key_id=''
S3_Secret_key=''

def pull_data(Key_id, Secret_key, file):
    """
    Function which CJ wrote to pull data from S3 
    """
    BUCKET_NAME = "gtown-wildfire-ds"
    OBJECT_KEY = file
    client = boto3.client(
        's3',
        aws_access_key_id= Key_id,
        aws_secret_access_key= Secret_key)
    obj = client.get_object(Bucket= BUCKET_NAME, Key= OBJECT_KEY) 
    file_df = pd.read_csv(obj['Body'])
    return (file_df)

#Pull in the firms and scan df
file = 'FIRMSandSCANFull2018toApr2022_V2.csv'
df_FirmsAndScan = pull_data(S3_Key_id, S3_Secret_key, file)
df_FirmsAndScan.head()

C:\Users\anderb4\AppData\Local\Temp\ipykernel_12796\109086010.py:17: DtypeWarning: Columns (9,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  file_df = pd.read_csv(obj['Body'])


,Unnamed: 0.1,Unnamed: 0,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_t31,frp,daynight,type,lat,long,hour,date_loc,date_hour_loc,newdate,nearestStation,StationDist,Date,Station Id,State Code,Network Code,Station Name,Elevation (ft),Latitude,Longitude,County Name,HUC2 Name,HUC4 Name,HUC6 Name,HUC8 Name,Precipitation Accumulation (in) Start of Day Values,Precipitation Increment (in),Precipitation Month-to-date (in) Start of Day Values,Air Temperature Average (degF),Soil Moisture Percent -2in (pct) Start of Day Values,Soil Moisture Percent -4in (pct) Start of Day Values,Relative Humidity Enclosure (pct),Wind Speed Average 324in (mph),Wind Speed Average 128in (mph),Wind Speed Average (mph),date_new,time,newdatetime,nearbydetections
0,0,0,33.4856,-90.2127,319.4,1.4,1.2,2018-01-01,427,Terra,MODIS,98,6.03,267.5,38.3,N,0.0,33.5,-90.2,4.0,"2018-01-01,33.5,-90.2","2018-01-01,04,33.5,-90.2",2018-01-01,Shaw Nature Reserve,38.496839,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,04:27,2018-01-01 04:27:00,0.0
1,1,1,44.3762,-119.1190,312.5,1.2,1.1,2018-01-01,608,Terra,MODIS,85,6.03,269.1,21.9,N,0.0,44.4,-119.1,6.0,"2018-01-01,44.4,-119.1","2018-01-01,06,44.4,-119.1",2018-01-01,Molly Caren #1,124.725193,2018-01-01,2014.0,OH,SCAN,Molly Caren #1,1060.0,39.95000,-83.45000,Madison,Ohio Region,Scioto,Scioto,Lower Scioto,11.5,0.0,NaN,2.0,36.1,36.4,78.0,NaN,NaN,7.1,2018-01-01,06:08,2018-01-01 06:08:00,0.0
2,2,2,19.4122,-155.2699,309.6,3.9,1.8,2018-01-01,919,Terra,MODIS,67,6.03,284.0,90.7,N,1.0,19.4,-155.3,9.0,"2018-01-01,19.4,-155.3","2018-01-01,09,19.4,-155.3",2018-01-01,Shaw Nature Reserve,39.500776,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,09:19,2018-01-01 09:19:00,5.0
3,3,3,19.3431,-155.0508,309.8,4.0,1.9,2018-01-01,919,Terra,MODIS,68,6.03,289.9,82.0,N,1.0,19.3,-155.1,9.0,"2018-01-01,19.3,-155.1","2018-01-01,09,19.3,-155.1",2018-01-01,Mason #1,170.344414,2018-01-01,2004.0,IL,SCAN,Mason #1,570.0,40.31667,-89.90000,Mason,Upper Mississippi Region,Lower Illinois,Lower Illinois,Lower Illinois-Lake Chautauqua,7.8,0.0,NaN,-8.0,4.9,6.2,66.0,NaN,NaN,1.8,2018-01-01,09:19,2018-01-01 09:19:00,5.0
4,4,4,19.3853,-155.0768,312.8,4.0,1.9,2018-01-01,919,Terra,MODIS,80,6.03,287.1,103.7,N,1.0,19.4,-155.1,9.0,"2018-01-01,19.4,-155.1","2018-01-01,09,19.4,-155.1",2018-01-01,Uapb-Earle,56.557018,2018-01-01,2085.0,AR,SCAN,Uapb-Earle,213.0,35.27732,-90.45031,Crittenden,Lower Mississippi Region,Lower Mississippi-St. Francis,St. Francis,Lower St. Francis,11.3,0.0,7.6,14.0,26.9,34.0,79.0,NaN,NaN,10.5,2018-01-01,09:19,2018-01-01 09:19:00,5.0


In [3]:
#pull in the big WFIGS dataframe
file = 'WFIGS_big_Pulled5-8-2022.csv'
df_wfigsbig = pull_data(S3_Key_id, S3_Secret_key, file)
df_wfigsbig.head()

C:\Users\anderb4\AppData\Local\Temp\ipykernel_12796\109086010.py:17: DtypeWarning: Columns (13,14,32,61,64,94,95) have mixed types. Specify dtype option on import or set low_memory=False.
  file_df = pd.read_csv(obj['Body'])


,X,Y,OBJECTID,ABCDMisc,ADSPermissionState,CalculatedAcres,ContainmentDateTime,ControlDateTime,DailyAcres,DiscoveryAcres,DispatchCenterID,EstimatedCostToDate,FinalFireReportApprovedByTitle,FinalFireReportApprovedByUnit,FinalFireReportApprovedDate,FireBehaviorGeneral,FireBehaviorGeneral1,FireBehaviorGeneral2,FireBehaviorGeneral3,FireCause,FireCauseGeneral,FireCauseSpecific,FireCode,FireDepartmentID,FireDiscoveryDateTime,FireMgmtComplexity,FireOutDateTime,FireStrategyConfinePercent,FireStrategyFullSuppPercent,FireStrategyMonitorPercent,FireStrategyPointZonePercent,FSJobCode,FSOverrideCode,GACC,ICS209ReportDateTime,ICS209ReportForTimePeriodFrom,ICS209ReportForTimePeriodTo,ICS209ReportStatus,IncidentManagementOrganization,IncidentName,IncidentShortDescription,IncidentTypeCategory,IncidentTypeKind,InitialLatitude,InitialLongitude,InitialResponseAcres,InitialResponseDateTime,IrwinID,IsFireCauseInvestigated,IsFireCodeRequested,IsFSAssisted,IsMultiJurisdictional,IsReimbursable,IsTrespass,IsUnifiedCommand,LocalIncidentIdentifier,PercentContained,PercentPerimeterToBeContained,POOCity,POOCounty,POODispatchCenterID,POOFips,POOJurisdictionalAgency,POOJurisdictionalUnit,POOJurisdictionalUnitParentUnit,POOLandownerCategory,POOLandownerKind,POOLegalDescPrincipalMeridian,POOLegalDescQtr,POOLegalDescQtrQtr,POOLegalDescRange,POOLegalDescSection,POOLegalDescTownship,POOPredictiveServiceAreaID,POOProtectingAgency,POOProtectingUnit,POOState,PredominantFuelGroup,PredominantFuelModel,PrimaryFuelModel,SecondaryFuelModel,TotalIncidentPersonnel,UniqueFireIdentifier,WFDSSDecisionStatus,CreatedBySystem,ModifiedBySystem,IsDispatchComplete,OrganizationalAssessment,StrategicDecisionPublishDate,CreatedOnDateTime_dt,ModifiedOnDateTime_dt,Source,GlobalID,IsCpxChild,CpxName,CpxID
0,-104.457511,45.785037,1,NaN,CERTIFIED,50.64,2020/08/06 23:13:07+00,2020/08/06 23:13:24+00,50.6,20.0,MTMCC,NaN,NaN,NaN,2020/08/12 20:45:59+00,NaN,NaN,NaN,NaN,Human,Equipment,Farming Equipment,NDU0,NaN,2020/08/06 18:58:00+00,NaN,2020/08/12 14:00:00+00,NaN,NaN,NaN,NaN,PN,1502,NRCC,NaN,NaN,NaN,NaN,NaN,Ramme,NaN,WF,FI,45.78496,-104.4958,NaN,NaN,{FE67856C-7401-44BD-B6F3-9A57EF990079},NaN,0,1.0,0.0,0.0,0.0,NaN,000224,NaN,NaN,NaN,Carter,MTMCC,30011.0,C&L,MTLG42,NaN,Private,Private,NaN,NaN,NaN,NaN,NaN,NaN,NR16,C&L,MTLG42,US-MT,Grass,GR4,NaN,NaN,NaN,2020-MTLG42-000224,No Decision,wildcad,INFORM_Inspector,0,NaN,NaN,2020/08/06 19:50:29+00,2020/08/12 20:46:01+00,IRWIN,{E5436898-ED0D-4CB1-90C0-D61915FE1F29},NaN,NaN,NaN
1,-118.180712,33.808985,2,NaN,DEFAULT,NaN,NaN,NaN,NaN,0.1,CALACC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unknown,NaN,NaN,NaN,NaN,2020/02/28 20:45:40+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,OSCC,NaN,NaN,NaN,NaN,NaN,LAC-066100,NaN,WF,FI,33.80898,-118.1807,NaN,NaN,{6A311ABB-DF4F-4947-B8DD-3900BDA784F6},NaN,0,NaN,NaN,NaN,NaN,NaN,066100,NaN,NaN,NaN,Los Angeles,CALACC,6037.0,NaN,CACAS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SC08,C&L,CALAC,US-CA,NaN,NaN,NaN,NaN,NaN,2020-CALAC-066100,No Decision,lacocad,lacocad,0,NaN,NaN,2020/02/28 20:52:36+00,2020/02/28 20:52:36+00,IRWIN,{0E79B7FD-2882-43CF-8CFA-911BD1C8F77A},NaN,NaN,NaN
2,-114.835414,48.073946,3,NaN,DEFAULT,NaN,2017/10/18 00:30:00+00,2017/10/18 00:35:00+00,50.0,50.0,MTKIC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Human,NaN,NaN,NaN,NaN,2017/10/17 20:20:24+00,NaN,2017/11/09 21:59:59+00,NaN,NaN,NaN,NaN,NaN,NaN,NRCC,NaN,NaN,NaN,NaN,NaN,South Lost Prairie,NaN,WF,FI,48.07167,-114.8303,1.0,NaN,{9599AE8E-66B8-4412-B849-7271271B0463},NaN,0,0.0,0.0,0.0,0.0,NaN,000878,NaN,NaN,NaN,Flathead,NaN,30029.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NR02,DNRC,MTNWS,US-MT,NaN,NaN,NaN,NaN,NaN,2017-MTNWS-000878,NaN,wildcad,wildcad,0,NaN,NaN,2017/10/18 13:46:40+00,2017/11/09 22:08:19+00,IRWIN,{FAC59A92-E6AD-443B-8625-4AAABCF7F533},NaN,NaN,NaN
3,-117.153901,33.176394,4,NaN,DEFAULT,NaN,NaN,NaN,NaN,NaN,CAMVIC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MC2S,NaN,2019/07/01 19:54:00+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,OSCC,NaN,NaN,NaN,NaN,NaN,SYCAMORE,NaN,WF,FI,NaN,NaN,NaN,NaN,{1AF2C949-B159-4

In [4]:
# now starting to transform the fire db
latlonground = 1 #Sets the number of decimal places to round latlong
#getting the fire discovered date into the same format as the same date
df_wfigsbig['disc_date'] = df_wfigsbig['FireDiscoveryDateTime'].astype(str).str[0:10]
df_wfigsbig['disc_date'] = df_wfigsbig['disc_date'].astype(str).str.replace('/', '-')
# #convert the init long and lat frame to 2 decimals like i did in the satellite df 
df_wfigsbig['init_lat_rounded'] = round(df_wfigsbig['Y'], latlonground) #rounds lat long into new column, using latlonground # of decimal pts
df_wfigsbig['init_long_rounded'] = round(df_wfigsbig['X'], latlonground) #rounds lat long into new column, using latlonground # of decimal pts
#make the date loc column
df_wfigsbig['disc_date_loc'] = df_wfigsbig['disc_date'] + ',' + df_wfigsbig['init_lat_rounded'].astype(str) + ',' + df_wfigsbig['init_long_rounded'].astype(str)
df_wfigsbig.head()

,X,Y,OBJECTID,ABCDMisc,ADSPermissionState,CalculatedAcres,ContainmentDateTime,ControlDateTime,DailyAcres,DiscoveryAcres,DispatchCenterID,EstimatedCostToDate,FinalFireReportApprovedByTitle,FinalFireReportApprovedByUnit,FinalFireReportApprovedDate,FireBehaviorGeneral,FireBehaviorGeneral1,FireBehaviorGeneral2,FireBehaviorGeneral3,FireCause,FireCauseGeneral,FireCauseSpecific,FireCode,FireDepartmentID,FireDiscoveryDateTime,FireMgmtComplexity,FireOutDateTime,FireStrategyConfinePercent,FireStrategyFullSuppPercent,FireStrategyMonitorPercent,FireStrategyPointZonePercent,FSJobCode,FSOverrideCode,GACC,ICS209ReportDateTime,ICS209ReportForTimePeriodFrom,ICS209ReportForTimePeriodTo,ICS209ReportStatus,IncidentManagementOrganization,IncidentName,IncidentShortDescription,IncidentTypeCategory,IncidentTypeKind,InitialLatitude,InitialLongitude,InitialResponseAcres,InitialResponseDateTime,IrwinID,IsFireCauseInvestigated,IsFireCodeRequested,IsFSAssisted,IsMultiJurisdictional,IsReimbursable,IsTrespass,IsUnifiedCommand,LocalIncidentIdentifier,PercentContained,PercentPerimeterToBeContained,POOCity,POOCounty,POODispatchCenterID,POOFips,POOJurisdictionalAgency,POOJurisdictionalUnit,POOJurisdictionalUnitParentUnit,POOLandownerCategory,POOLandownerKind,POOLegalDescPrincipalMeridian,POOLegalDescQtr,POOLegalDescQtrQtr,POOLegalDescRange,POOLegalDescSection,POOLegalDescTownship,POOPredictiveServiceAreaID,POOProtectingAgency,POOProtectingUnit,POOState,PredominantFuelGroup,PredominantFuelModel,PrimaryFuelModel,SecondaryFuelModel,TotalIncidentPersonnel,UniqueFireIdentifier,WFDSSDecisionStatus,CreatedBySystem,ModifiedBySystem,IsDispatchComplete,OrganizationalAssessment,StrategicDecisionPublishDate,CreatedOnDateTime_dt,ModifiedOnDateTime_dt,Source,GlobalID,IsCpxChild,CpxName,CpxID,disc_date,init_lat_rounded,init_long_rounded,disc_date_loc
0,-104.457511,45.785037,1,NaN,CERTIFIED,50.64,2020/08/06 23:13:07+00,2020/08/06 23:13:24+00,50.6,20.0,MTMCC,NaN,NaN,NaN,2020/08/12 20:45:59+00,NaN,NaN,NaN,NaN,Human,Equipment,Farming Equipment,NDU0,NaN,2020/08/06 18:58:00+00,NaN,2020/08/12 14:00:00+00,NaN,NaN,NaN,NaN,PN,1502,NRCC,NaN,NaN,NaN,NaN,NaN,Ramme,NaN,WF,FI,45.78496,-104.4958,NaN,NaN,{FE67856C-7401-44BD-B6F3-9A57EF990079},NaN,0,1.0,0.0,0.0,0.0,NaN,000224,NaN,NaN,NaN,Carter,MTMCC,30011.0,C&L,MTLG42,NaN,Private,Private,NaN,NaN,NaN,NaN,NaN,NaN,NR16,C&L,MTLG42,US-MT,Grass,GR4,NaN,NaN,NaN,2020-MTLG42-000224,No Decision,wildcad,INFORM_Inspector,0,NaN,NaN,2020/08/06 19:50:29+00,2020/08/12 20:46:01+00,IRWIN,{E5436898-ED0D-4CB1-90C0-D61915FE1F29},NaN,NaN,NaN,2020-08-06,45.8,-104.5,"2020-08-06,45.8,-104.5"
1,-118.180712,33.808985,2,NaN,DEFAULT,NaN,NaN,NaN,NaN,0.1,CALACC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unknown,NaN,NaN,NaN,NaN,2020/02/28 20:45:40+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,OSCC,NaN,NaN,NaN,NaN,NaN,LAC-066100,NaN,WF,FI,33.80898,-118.1807,NaN,NaN,{6A311ABB-DF4F-4947-B8DD-3900BDA784F6},NaN,0,NaN,NaN,NaN,NaN,NaN,066100,NaN,NaN,NaN,Los Angeles,CALACC,6037.0,NaN,CACAS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SC08,C&L,CALAC,US-CA,NaN,NaN,NaN,NaN,NaN,2020-CALAC-066100,No Decision,lacocad,lacocad,0,NaN,NaN,2020/02/28 20:52:36+00,2020/02/28 20:52:36+00,IRWIN,{0E79B7FD-2882-43CF-8CFA-911BD1C8F77A},NaN,NaN,NaN,2020-02-28,33.8,-118.2,"2020-02-28,33.8,-118.2"
2,-114.835414,48.073946,3,NaN,DEFAULT,NaN,2017/10/18 00:30:00+00,2017/10/18 00:35:00+00,50.0,50.0,MTKIC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Human,NaN,NaN,NaN,NaN,2017/10/17 20:20:24+00,NaN,2017/11/09 21:59:59+00,NaN,NaN,NaN,NaN,NaN,NaN,NRCC,NaN,NaN,NaN,NaN,NaN,South Lost Prairie,NaN,WF,FI,48.07167,-114.8303,1.0,NaN,{9599AE8E-66B8-4412-B849-7271271B0463},NaN,0,0.0,0.0,0.0,0.0,NaN,000878,NaN,NaN,NaN,Flathead,NaN,30029.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NR02,DNRC,MTNWS,US-MT,NaN,NaN,NaN,NaN,NaN,2017-MTNWS-000878,NaN,wildcad,wildcad,0,NaN,NaN,2017/10/18 13:46:40+00,2017/11/09 22:08:19+00,IRWIN,{FAC59A92-E6AD-443B-8625-4AAABCF7F533},NaN,NaN,NaN,2017-10-17,48.1,-114.8,"2017-10-17,48.1,-114.8"
3,-117.153901,33.176394,4,NaN,DEFAULT,NaN,NaN,

In [5]:
#Merging on the date + location key.
dfDiscovered = df_FirmsAndScan.merge(df_wfigsbig, left_on='date_loc', right_on='disc_date_loc', how='inner') #creates df of only dsetected areas
dfMerged = df_FirmsAndScan.merge(df_wfigsbig, left_on='date_loc', right_on='disc_date_loc', how='left') #creates df of both 
dfMerged['FIRE_DETECTED'] = dfMerged['OBJECTID'].isnull() #if there was no fire detectection set to true
dfMerged['FIRE_DETECTED'] = ~dfMerged['FIRE_DETECTED'].astype(bool) #flip so that if detection merged set to true
print(dfMerged.shape)
dfMerged.head()

(2096070, 153)


,Unnamed: 0.1,Unnamed: 0,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_t31,frp,daynight,type,lat,long,hour,date_loc,date_hour_loc,newdate,nearestStation,StationDist,Date,Station Id,State Code,Network Code,Station Name,Elevation (ft),Latitude,Longitude,County Name,HUC2 Name,HUC4 Name,HUC6 Name,HUC8 Name,Precipitation Accumulation (in) Start of Day Values,Precipitation Increment (in),Precipitation Month-to-date (in) Start of Day Values,Air Temperature Average (degF),Soil Moisture Percent -2in (pct) Start of Day Values,Soil Moisture Percent -4in (pct) Start of Day Values,Relative Humidity Enclosure (pct),Wind Speed Average 324in (mph),Wind Speed Average 128in (mph),Wind Speed Average (mph),date_new,time,newdatetime,nearbydetections,X,Y,OBJECTID,ABCDMisc,ADSPermissionState,CalculatedAcres,ContainmentDateTime,ControlDateTime,DailyAcres,DiscoveryAcres,DispatchCenterID,EstimatedCostToDate,FinalFireReportApprovedByTitle,FinalFireReportApprovedByUnit,FinalFireReportApprovedDate,FireBehaviorGeneral,FireBehaviorGeneral1,FireBehaviorGeneral2,FireBehaviorGeneral3,FireCause,FireCauseGeneral,FireCauseSpecific,FireCode,FireDepartmentID,FireDiscoveryDateTime,FireMgmtComplexity,FireOutDateTime,FireStrategyConfinePercent,FireStrategyFullSuppPercent,FireStrategyMonitorPercent,FireStrategyPointZonePercent,FSJobCode,FSOverrideCode,GACC,ICS209ReportDateTime,ICS209ReportForTimePeriodFrom,ICS209ReportForTimePeriodTo,ICS209ReportStatus,IncidentManagementOrganization,IncidentName,IncidentShortDescription,IncidentTypeCategory,IncidentTypeKind,InitialLatitude,InitialLongitude,InitialResponseAcres,InitialResponseDateTime,IrwinID,IsFireCauseInvestigated,IsFireCodeRequested,IsFSAssisted,IsMultiJurisdictional,IsReimbursable,IsTrespass,IsUnifiedCommand,LocalIncidentIdentifier,PercentContained,PercentPerimeterToBeContained,POOCity,POOCounty,POODispatchCenterID,POOFips,POOJurisdictionalAgency,POOJurisdictionalUnit,POOJurisdictionalUnitParentUnit,POOLandownerCategory,POOLandownerKind,POOLegalDescPrincipalMeridian,POOLegalDescQtr,POOLegalDescQtrQtr,POOLegalDescRange,POOLegalDescSection,POOLegalDescTownship,POOPredictiveServiceAreaID,POOProtectingAgency,POOProtectingUnit,POOState,PredominantFuelGroup,PredominantFuelModel,PrimaryFuelModel,SecondaryFuelModel,TotalIncidentPersonnel,UniqueFireIdentifier,WFDSSDecisionStatus,CreatedBySystem,ModifiedBySystem,IsDispatchComplete,OrganizationalAssessment,StrategicDecisionPublishDate,CreatedOnDateTime_dt,ModifiedOnDateTime_dt,Source,GlobalID,IsCpxChild,CpxName,CpxID,disc_date,init_lat_rounded,init_long_rounded,disc_date_loc,FIRE_DETECTED
0,0,0,33.4856,-90.2127,319.4,1.4,1.2,2018-01-01,427,Terra,MODIS,98,6.03,267.5,38.3,N,0.0,33.5,-90.2,4.0,"2018-01-01,33.5,-90.2","2018-01-01,04,33.5,-90.2",2018-01-01,Shaw Nature Reserve,38.496839,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,04:27,2018-01-01 04:27:00,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
1,1,1,44.3762,-119.1190,312.5,1.2,1.1,2018-01-01,608,Terra,MODIS,85,6.03,269.1,21.9,N,0.0,44.4,-119.1,6.0,"2018-01-01,44.4,-119.1","2018-01-01,06,44.4,-119.1",2018-01-01,Molly Caren #1,124.725193,2018-01-01,2014.0,OH,SCAN,Molly Caren #1,1060.0,39.95000,-83.45000,Madison,Ohio Region,Scioto,Scioto,Lower Scioto,11.5,0.0,NaN,2.0,36.1,36.4,78.0,NaN,NaN,7.1,2018-01-01,06:08,2018-01-01 06:08:00,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [6]:
#check the number of merges
dfDiscovered.shape

(63846, 152)

## Clean and Prep the Data

In [7]:
#beign cleaning the data by removing unnecessary columns:
#drop the columns we VERY likely do not need for this approach
coltodrop = ['Unnamed: 0',
'Unnamed: 0.1',
'acq_date',
'acq_time',
'version',
'lat',
'long',
'hour',
'date_loc',
'date_hour_loc',
'newdate',
'nearestStation',
'Date',
'Station Id',
'State Code',
'Network Code',
'Station Name',
'Elevation (ft)',
'Latitude',
'Longitude',
'County Name',
'HUC6 Name',
'HUC8 Name',
'Wind Speed Average 324in (mph)',
'Wind Speed Average 128in (mph)',
'date_new',
'X',
'Y',
'OBJECTID',
'ABCDMisc',
'ADSPermissionState',
'CalculatedAcres',
'ContainmentDateTime',
'ControlDateTime',
'DailyAcres',
'DiscoveryAcres',
'DispatchCenterID',
'EstimatedCostToDate',
'FinalFireReportApprovedByTitle',
'FinalFireReportApprovedByUnit',
'FinalFireReportApprovedDate',
'FireBehaviorGeneral',
'FireBehaviorGeneral1',
'FireBehaviorGeneral2',
'FireBehaviorGeneral3',
'FireCause',
'FireCauseGeneral',
'FireCauseSpecific',
'FireCode',
'FireDepartmentID',
'FireDiscoveryDateTime',
'FireMgmtComplexity',
'FireOutDateTime',
'FireStrategyConfinePercent',
'FireStrategyFullSuppPercent',
'FireStrategyMonitorPercent',
'FireStrategyPointZonePercent',
'FSJobCode',
'FSOverrideCode',
'GACC',
'ICS209ReportDateTime',
'ICS209ReportForTimePeriodFrom',
'ICS209ReportForTimePeriodTo',
'ICS209ReportStatus',
'IncidentManagementOrganization',
'IncidentName',
'IncidentShortDescription',
'IncidentTypeCategory',
'IncidentTypeKind',
'InitialLatitude',
'InitialLongitude',
'InitialResponseAcres',
'InitialResponseDateTime',
'IrwinID',
'IsFireCauseInvestigated',
'IsFireCodeRequested',
'IsFSAssisted',
'IsMultiJurisdictional',
'IsReimbursable',
'IsTrespass',
'IsUnifiedCommand',
'LocalIncidentIdentifier',
'PercentContained',
'PercentPerimeterToBeContained',
'POOCity',
'POOCounty',
'POODispatchCenterID',
'POOFips',
'POOJurisdictionalAgency',
'POOJurisdictionalUnit',
'POOJurisdictionalUnitParentUnit',
'POOLandownerCategory',
'POOLandownerKind',
'POOLegalDescPrincipalMeridian',
'POOLegalDescQtr',
'POOLegalDescQtrQtr',
'POOLegalDescRange',
'POOLegalDescSection',
'POOLegalDescTownship',
'POOPredictiveServiceAreaID',
'POOProtectingAgency',
'POOProtectingUnit',
'POOState',
'PredominantFuelGroup',
'PredominantFuelModel',
'PrimaryFuelModel',
'SecondaryFuelModel',
'TotalIncidentPersonnel',
'UniqueFireIdentifier',
'WFDSSDecisionStatus',
'CreatedBySystem',
'ModifiedBySystem',
'IsDispatchComplete',
'OrganizationalAssessment',
'StrategicDecisionPublishDate',
'CreatedOnDateTime_dt',
'ModifiedOnDateTime_dt',
'Source',
'GlobalID',
'IsCpxChild',
'CpxName',
'CpxID',
'disc_date',
'init_lat_rounded',
'init_long_rounded',
'disc_date_loc',
'time',
'newdatetime'
]
dfMerged = dfMerged.drop(coltodrop, axis = 1) #drops unwanted columns for ML process
dfMerged.head()

,latitude,longitude,brightness,scan,track,satellite,instrument,confidence,bright_t31,frp,daynight,type,StationDist,HUC2 Name,HUC4 Name,Precipitation Accumulation (in) Start of Day Values,Precipitation Increment (in),Precipitation Month-to-date (in) Start of Day Values,Air Temperature Average (degF),Soil Moisture Percent -2in (pct) Start of Day Values,Soil Moisture Percent -4in (pct) Start of Day Values,Relative Humidity Enclosure (pct),Wind Speed Average (mph),nearbydetections,FIRE_DETECTED
0,33.4856,-90.2127,319.4,1.4,1.2,Terra,MODIS,98,267.5,38.3,N,0.0,38.496839,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,False
1,44.3762,-119.1190,312.5,1.2,1.1,Terra,MODIS,85,269.1,21.9,N,0.0,124.725193,Ohio Region,Scioto,11.5,0.0,NaN,2.0,36.1,36.4,78.0,7.1,0.0,False
2,19.4122,-155.2699,309.6,3.9,1.8,Terra,MODIS,67,284.0,90.7,N,1.0,39.500776,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,False
3,19.3431,-155.0508,309.8,4.0,1.9,Terra,MODIS,68,289.9,82.0,N,1.0,170.344414,Upper Mississippi Region,Lower Illinois,7.8,0.0,NaN,-8.0,4.9,6.2,66.0,1.8,5.0,False
4,19.3853,-155.0768,312.8,4.0,1.9,Terra,MODIS,80,287.1,103.7,N,1.0,56.557018,Lower Mississippi Region,Lower Mississippi-St. Francis,11.3,0.0,7.6,14.0,26.9,34.0,79.0,10.5,5.0,False


In [8]:
#remove columns we (likely) dont need for this approach 
coltodrop = ['latitude',
                'longitude',
                'daynight',
                'Soil Moisture Percent -4in (pct) Start of Day Values',
                'HUC4 Name']
dfMerged = dfMerged.drop(coltodrop, axis = 1) #drops unwanted columns for ML process
dfMerged.head()

,brightness,scan,track,satellite,instrument,confidence,bright_t31,frp,type,StationDist,HUC2 Name,Precipitation Accumulation (in) Start of Day Values,Precipitation Increment (in),Precipitation Month-to-date (in) Start of Day Values,Air Temperature Average (degF),Soil Moisture Percent -2in (pct) Start of Day Values,Relative Humidity Enclosure (pct),Wind Speed Average (mph),nearbydetections,FIRE_DETECTED
0,319.4,1.4,1.2,Terra,MODIS,98,267.5,38.3,0.0,38.496839,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,False
1,312.5,1.2,1.1,Terra,MODIS,85,269.1,21.9,0.0,124.725193,Ohio Region,11.5,0.0,NaN,2.0,36.1,78.0,7.1,0.0,False
2,309.6,3.9,1.8,Terra,MODIS,67,284.0,90.7,1.0,39.500776,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,False
3,309.8,4.0,1.9,Terra,MODIS,68,289.9,82.0,1.0,170.344414,Upper Mississippi Region,7.8,0.0,NaN,-8.0,4.9,66.0,1.8,5.0,False
4,312.8,4.0,1.9,Terra,MODIS,80,287.1,103.7,1.0,56.557018,Lower Mississippi Region,11.3,0.0,7.6,14.0,26.9,79.0,10.5,5.0,False


In [9]:
#checking some countsd of values on missing columns 
collist = list(dfMerged.columns) #get the list of columns in the df
numbrows = dfMerged.shape[0] #saves number of riows in the df
for col in collist:
    missingcount = dfMerged[col].isnull().sum() #count missing vals 
    missingpercent = round((missingcount/numbrows)*100, 2) #gets the percent compared to the entire dataframe
    print('number of nan for '+col+':'+str(missingcount)+' ; '+str(missingpercent)+'%') #prints the result

number of nan for brightness:0 ; 0.0%
number of nan for scan:0 ; 0.0%
number of nan for track:0 ; 0.0%
number of nan for satellite:0 ; 0.0%
number of nan for instrument:0 ; 0.0%
number of nan for confidence:0 ; 0.0%
number of nan for bright_t31:0 ; 0.0%
number of nan for frp:0 ; 0.0%
number of nan for type:1542889 ; 73.61%
number of nan for StationDist:0 ; 0.0%
number of nan for HUC2 Name:51886 ; 2.48%
number of nan for Precipitation Accumulation (in) Start of Day Values:479812 ; 22.89%
number of nan for Precipitation Increment (in):482593 ; 23.02%
number of nan for Precipitation Month-to-date (in) Start of Day Values:816760 ; 38.97%
number of nan for Air Temperature Average (degF):231959 ; 11.07%
number of nan for Soil Moisture Percent -2in (pct) Start of Day Values:400189 ; 19.09%
number of nan for Relative Humidity Enclosure (pct):185163 ; 8.83%
number of nan for Wind Speed Average (mph):185817 ; 8.87%
number of nan for nearbydetections:0 ; 0.0%
number of nan for FIRE_DETECTED:0 ; 0

In [10]:
#there appears to be a lot of missing data for 'type' and 'precipitation month to date'. drop those for this approach
dfMerged = dfMerged.drop('type', axis = 1)
dfMerged = dfMerged.drop('Precipitation Month-to-date (in) Start of Day Values', axis = 1)
#the station distance is not relevant for a fire prediction so dropping
#we may want to drop rows where the nearest station distance is too far in the future
dfMerged = dfMerged.drop('StationDist', axis = 1)
dfMerged.head()

,brightness,scan,track,satellite,instrument,confidence,bright_t31,frp,HUC2 Name,Precipitation Accumulation (in) Start of Day Values,Precipitation Increment (in),Air Temperature Average (degF),Soil Moisture Percent -2in (pct) Start of Day Values,Relative Humidity Enclosure (pct),Wind Speed Average (mph),nearbydetections,FIRE_DETECTED
0,319.4,1.4,1.2,Terra,MODIS,98,267.5,38.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,False
1,312.5,1.2,1.1,Terra,MODIS,85,269.1,21.9,Ohio Region,11.5,0.0,2.0,36.1,78.0,7.1,0.0,False
2,309.6,3.9,1.8,Terra,MODIS,67,284.0,90.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,False
3,309.8,4.0,1.9,Terra,MODIS,68,289.9,82.0,Upper Mississippi Region,7.8,0.0,-8.0,4.9,66.0,1.8,5.0,False
4,312.8,4.0,1.9,Terra,MODIS,80,287.1,103.7,Lower Mississippi Region,11.3,0.0,14.0,26.9,79.0,10.5,5.0,False


In [11]:
#'confidence' appears to be starnge. based off the link below: n=nominal l = low and h = high
# https://cdn.earthdata.nasa.gov/conduit/upload/10575/MODIS_C6_Fire_User_Guide_B.pdf
#another helpful link:
# https://earthdata.nasa.gov/faq/firms-faq#ed-confidence
# lets approximate low 20, nominal 50, and high 80 for this table 
# this can be adjusted later, if needed
mapping = {'l': 20, 'n': 50, 'h': 100}
dfMerged = dfMerged.replace({'confidence': mapping})
dfMerged['confidence'] = pd.to_numeric(dfMerged['confidence']) #convert confidence from string to integer

In [12]:
#now drop the nan rows 
print('df shape before dropping all nan: '+str(dfMerged.shape))
dfMerged = dfMerged.dropna() #drop all nan values 
print('df shape after dropping all nan: '+str(dfMerged.shape))

df shape before dropping all nan: (2096070, 17)
df shape after dropping all nan: (1396691, 17)


In [13]:
#check how many trues we have
dfMerged['FIRE_DETECTED'].value_counts()

False    1357647
True       39044
Name: FIRE_DETECTED, dtype: int64

In [14]:
#now begin to one hot encode the 'satellite' column
#first check what kind of values we have for the satellite column
dfMerged['satellite'].value_counts()

1        965004
Aqua     217886
Terra    207087
1          6714
Name: satellite, dtype: int64

In [15]:
#weird how '1' appears twice, convert to list to check why and if different datatype
dfMerged['satellite'].unique().tolist()

['Terra', 'Aqua', '1', 1]

In [16]:
#looks like some 1's are strings and some are ints. converting everything to string
dfMerged['satellite'] = dfMerged['satellite'].astype(str)

In [17]:
#check to make sure we are good now
dfMerged['satellite'].value_counts()

1        971718
Aqua     217886
Terra    207087
Name: satellite, dtype: int64

In [18]:
#add the one hot encoded df to the merged dataframe and drop the original satellite column
sat_dummies = pd.get_dummies(dfMerged.satellite)
dfMerged = pd.concat([dfMerged, sat_dummies], axis=1) #adds in the one hot encoded  staellite df
dfMerged = dfMerged.drop('satellite', axis = 1) #drop the satellite column
dfMerged.head()

,brightness,scan,track,instrument,confidence,bright_t31,frp,HUC2 Name,Precipitation Accumulation (in) Start of Day Values,Precipitation Increment (in),Air Temperature Average (degF),Soil Moisture Percent -2in (pct) Start of Day Values,Relative Humidity Enclosure (pct),Wind Speed Average (mph),nearbydetections,FIRE_DETECTED,1,Aqua,Terra
1,312.5,1.2,1.1,MODIS,85,269.1,21.9,Ohio Region,11.5,0.0,2.0,36.1,78.0,7.1,0.0,False,0,0,1
3,309.8,4.0,1.9,MODIS,68,289.9,82.0,Upper Mississippi Region,7.8,0.0,-8.0,4.9,66.0,1.8,5.0,False,0,0,1
4,312.8,4.0,1.9,MODIS,80,287.1,103.7,Lower Mississippi Region,11.3,0.0,14.0,26.9,79.0,10.5,5.0,False,0,0,1
5,310.2,4.0,1.9,MODIS,70,287.3,83.6,Lower Mississippi Region,8.4,0.0,11.0,8.0,88.0,4.2,5.0,False,0,0,1
10,300.4,1.0,1.0,MODIS,28,281.6,6.3,Tennessee Region,12.8,0.0,15.0,33.1,88.0,3.6,23.0,False,0,1,0


In [19]:
#now one hot encode instrument
ohe = pd.get_dummies(dfMerged.instrument) #creates the one hot encoded df of instrument
dfMerged = pd.concat([dfMerged, ohe], axis=1) #merges the dataframe with the one hot encoded df
dfMerged = dfMerged.drop('instrument', axis = 1) #drop the instrument column
dfMerged.head()

,brightness,scan,track,confidence,bright_t31,frp,HUC2 Name,Precipitation Accumulation (in) Start of Day Values,Precipitation Increment (in),Air Temperature Average (degF),Soil Moisture Percent -2in (pct) Start of Day Values,Relative Humidity Enclosure (pct),Wind Speed Average (mph),nearbydetections,FIRE_DETECTED,1,Aqua,Terra,MODIS,VIIRS
1,312.5,1.2,1.1,85,269.1,21.9,Ohio Region,11.5,0.0,2.0,36.1,78.0,7.1,0.0,False,0,0,1,1,0
3,309.8,4.0,1.9,68,289.9,82.0,Upper Mississippi Region,7.8,0.0,-8.0,4.9,66.0,1.8,5.0,False,0,0,1,1,0
4,312.8,4.0,1.9,80,287.1,103.7,Lower Mississippi Region,11.3,0.0,14.0,26.9,79.0,10.5,5.0,False,0,0,1,1,0
5,310.2,4.0,1.9,70,287.3,83.6,Lower Mississippi Region,8.4,0.0,11.0,8.0,88.0,4.2,5.0,False,0,0,1,1,0
10,300.4,1.0,1.0,28,281.6,6.3,Tennessee Region,12.8,0.0,15.0,33.1,88.0,3.6,23.0,False,0,1,0,1,0


In [20]:
#because there is only 2 instruments (MODIS and VIIRS) we can drop one of the columns 
#we will keep the MODIS column, so 1 = MODIS and 0 = VIIRS instrument
dfMerged = dfMerged.drop('VIIRS', axis = 1) #drop the instrument column

In [21]:
#the column name 'HUC2 Name' will not work for the necessary syntax because it has a space in it. 
dfMerged = dfMerged.rename(columns={"HUC2 Name": "HUC2Name"}) #renames the HUC2 Name column to remove spaces 
#now one hot encode the region by HUC2 name:
ohe = pd.get_dummies(dfMerged.HUC2Name) #creates the one hot encoded df of instrument
dfMerged = pd.concat([dfMerged, ohe], axis=1) #merges the dataframe with the one hot encoded df
dfMerged = dfMerged.drop('HUC2Name', axis = 1) #drop the instrument column
dfMerged.head()

,brightness,scan,track,confidence,bright_t31,frp,Precipitation Accumulation (in) Start of Day Values,Precipitation Increment (in),Air Temperature Average (degF),Soil Moisture Percent -2in (pct) Start of Day Values,Relative Humidity Enclosure (pct),Wind Speed Average (mph),nearbydetections,FIRE_DETECTED,1,Aqua,Terra,MODIS,Arkansas-White-Red Region,California Region,Great Basin Region,Great Lakes Region,Hawaii Region,Lower Colorado Region,Lower Mississippi Region,Mid Atlantic Region,Missouri Region,New England Region,Ohio Region,Pacific Northwest Region,Rio Grande Region,Souris-Red-Rainy Region,South Atlantic-Gulf Region,Tennessee Region,Texas-Gulf Region,Upper Colorado Region,Upper Mississippi Region
1,312.5,1.2,1.1,85,269.1,21.9,11.5,0.0,2.0,36.1,78.0,7.1,0.0,False,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
3,309.8,4.0,1.9,68,289.9,82.0,7.8,0.0,-8.0,4.9,66.0,1.8,5.0,False,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,312.8,4.0,1.9,80,287.1,103.7,11.3,0.0,14.0,26.9,79.0,10.5,5.0,False,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
5,310.2,4.0,1.9,70,287.3,83.6,8.4,0.0,11.0,8.0,88.0,4.2,5.0,False,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
10,300.4,1.0,1.0,28,281.6,6.3,12.8,0.0,15.0,33.1,88.0,3.6,23.0,False,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [22]:
#check the data types to make sure they are all values and not strings
dfMerged.dtypes

brightness                                              float64
scan                                                    float64
track                                                   float64
confidence                                                int64
bright_t31                                              float64
frp                                                     float64
Precipitation Accumulation (in) Start of Day Values     float64
Precipitation Increment (in)                            float64
Air Temperature Average (degF)                          float64
Soil Moisture Percent -2in (pct) Start of Day Values    float64
Relative Humidity Enclosure (pct)                       float64
Wind Speed Average (mph)                                float64
nearbydetections                                        float64
FIRE_DETECTED                                              bool
1                                                         uint8
Aqua                                    

### Save the data 

In [24]:
dfMerged.to_csv('MLTable1.csv')